# Read a DEM

In [2]:
%matplotlib

import numpy as np
import landlab

Using matplotlib backend: TkAgg


In [3]:
grid, z = landlab.io.read_esri_ascii("data/puertorico_dem90fil.asc", name="topographic__elevation")
# grid, z = landlab.io.read_esri_ascii("puerto_rico.asc", name="topographic__elevation")

grid.at_node.keys()

dict_keys(['topographic__elevation'])

In [4]:
ocean = grid.at_node["topographic__elevation"] < 0.
grid.at_node["topographic__elevation"][ocean] = -999.

grid.set_nodata_nodes_to_closed(z, nodata_value=-999.)

In [5]:
landlab.plot.imshow_grid(grid, "topographic__elevation", cmap="terrain")

# Find watersheds

In [6]:
from landlab.utils import get_watershed_masks_with_area_threshold
from landlab.components import FlowAccumulator

In [23]:
flow_accumulator = FlowAccumulator(grid, flow_director='D8')# , depression_finder='DepressionFinderAndRouter')
flow_accumulator.run_one_step()

In [24]:
watersheds = get_watershed_masks_with_area_threshold(grid, 8e4)

In [13]:
def rand_cmap(n_values, select=None):
    import matplotlib.pyplot as plt
    import numpy as np
    
    low, high = 0.6, 0.95
    if select is None:
        colors = np.random.uniform(low=low, high=high, size=n_values * 3).reshape((-1, 3))
    else:
        colors = np.zeros(n_values * 3).reshape((-1, 3))
        colors[select] = np.random.uniform(low=low, high=high, size=len(select)).reshape((-1, 3))

    cmap = plt.cm.colors.ListedColormap(colors)
    cmap.set_under('white')
    
    return cmap

In [29]:
landlab.plot.imshow_grid(
    grid,
    watersheds,
    cmap=rand_cmap(watersheds.max()),
)


Grid query results:

{'grid location': {'x_coord': 195755.35282258061,
                   'y_coord': 131236.42137096776},
 'node': {'ID': 5224731,
          'column': 2175,
          'row': 1458}}


## Read sample locations

In [30]:
import pandas

In [31]:
data = pandas.read_csv("data/sample_locations.txt")

In [32]:
data

,FID,Count,System,Sample,SampleID,Latitude,Longitude,GeoNetwork
0,0,2.0,D_1,Res,D_1_Res,18.033150,-65.981430,Moved 200m downstream to link upstream of D_2
1,1,14.0,D_2,Res,D_2_Res,18.030945,-65.980139,No change
2,2,24.0,F,Res,F_Res,18.033158,-65.988490,Moved downhill to north network 300 meters
3,3,37.0,B,URes,B_URes,18.077463,-66.051863,Moved douth 100 m - uncertain on side of ridge
4,4,38.0,B,LRes,B_LRes,18.076606,-66.053434,Moved 100m north onto network
5,5,50.0,A,Res,A_Res,18.063904,-66.029455,No change
6,6,63.0,AB,1,AB_1,18.259383,-66.103466,No change
7,7,64.0,AB,2,AB_2,18.258465,-66.094181,No change
8,8,65.0,ABCG,3,ABCG_3,18.256978,-66.065114,No change
9,9,66.0,CG,4,CG_4,18.259921,-66.026112,No change


In [158]:
import pyproj

In [159]:
p = pyproj.Proj('+proj=utm +zone=20N +ellps=WGS84')

In [160]:
xy_of_location = p(data["Longitude"].values, data["Latitude"].values)

In [162]:
id_of_location = grid.find_nearest_node(p2(data["Longitude"].values, data["Latitude"].values))

In [163]:
grid.at_node["topographic__elevation"][id_of_location]

array([ 460.525   ,  401.6872  ,  461.0269  ,  386.8622  ,  336.8474  ,
        253.0092  ,  243.0343  ,  211.5359  ,   86.92188 ,   53.07581 ,
         47.18911 ,   97.28863 ,   25.48574 ,   10.59097 ,    9.374895,
        311.4903  ,  101.6129  ,   72.51905 ,  276.0992  ,  210.8874  ,
        171.8296  ,  110.554   ,  106.6357  ,  101.6953  ,   64.15204 ,
         64.26633 ])

In [164]:
watersheds[id_of_location]

array([4605284, 4508562, 4576589, 4802206, 4802206, 4737758, 5615272,
       5615280, 5604575, 5611778, 5611784, 4508562, 4393911, 4393911,
       4393911, 4669725, 4576589, 4576589, 4802206, 4773549, 4752065,
       4676857, 4648204, 4594477, 4576570, 4555092])

In [168]:
def cmap(n_values, values):
    import matplotlib.pyplot as plt
    import numpy as np
    
    low, high = 0.6, 0.95
    colors = np.zeros(n_values * 3).reshape((-1, 3))
    colors[values] = np.random.uniform(low=low, high=high, size=len(values) * 3).reshape((-1, 3))

    cmap = plt.cm.colors.ListedColormap(colors)
    cmap.set_under('white')
    
    return cmap

In [171]:
landlab.plot.imshow_grid(
    grid,
    watersheds,
    cmap=rand_cmap(watersheds.max(), watersheds[id_of_location]),
)


Grid query results:

{'grid location': {'x_coord': 189255.75604838709,
                   'y_coord': 150735.21169354836},
 'node': {'ID': 6001953,
          'column': 2103,
          'row': 1675}}
